In [ ]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenent/oauth2/token"}


dbutils.fs.mount(
source = "abfss://container@storagacc.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/people",
extra_configs = configs)

In [ ]:
%fs
ls "/mnt/people/"

In [ ]:
people=spark.read.csv("dbfs:/mnt/people/People.csv",header=True,inferSchema=True)
organizations=spark.read.csv("dbfs:/mnt/people/organizations.csv",header=True,inferSchema=True)
positions=spark.read.csv("dbfs:/mnt/people/positions.csv",header=True,inferSchema=True)

In [ ]:
display(people)

In [ ]:
display(positions)

In [ ]:
display(organizations)

In [ ]:
positions=positions.withColumnRenamed('department_ID','org_ID')

In [ ]:
pos_org=positions.join(organizations,on="org_ID")

In [ ]:
display(pos_org)

In [ ]:
pos_org_peop=pos_org.join(people,on="person_ID")

In [ ]:
display(pos_org_peop)

In [ ]:
people.count()

In [ ]:
no_prof_by_depart=pos_org_peop.groupBy('title').count()

In [ ]:
no_prof_by_depart.show()

In [ ]:
people.repartition(1).format("com.databricks.spark.csv").write.csv("dbfs:/mnt/people/transfromed_data/People",mode="overwrite",header=True,compression='none')
organizations.repartition(1).write.csv("dbfs:/mnt/people/transfromed_data/organizations",mode="overwrite",header=True,compression='none')
positions.repartition(1).write.csv("dbfs:/mnt/people/transfromed_data/positions",mode="overwrite",header=True,compression='none')